# Importing required libraries

In [1]:
from dotenv import load_dotenv,find_dotenv

In [2]:
import pandas as pd

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from langchain.agents import create_pandas_dataframe_agent

In [5]:
import json

In [27]:
import openai
openai.api_type = "azure"
openai.api_key = "" # replace with your key
openai.api_base = "https://{yourendpoint}.openai.azure.com/" # replace with your api_base
openai.api_version = "2023-03-15-preview"
deployment_id = 'gpt-4'

# Creating system message for OpenAI Chat Completion API

This will guarantee OpenAI understands the meaning of each column the dataframe I will read later

In [7]:
system_message = """Your name is Pepe and you are a data anaytics engineer. Your domain knowledge is about python, Pandas, and analytics. If you are asked about anything else, you reply "N/A". If you do not have enought information, reply "N/A" 
Description of some numeric columns the data frame:
"Unit price" is the price per unit 
"Quantity" is the number of items purchased 
"cogs" is the product of "unit price" and "Quantity"
"Tax 5%" is the tax calculated over the "cogs"
"Total" is the sum of "cogs" plus "Tax 5%"
"""

# Creating an instance of the Langchain chat

This command will pass its parameters to Azure OpenAI gtp-4 model

In [8]:
chat=ChatOpenAI(temperature=0.0,engine="gpt-4")
#chat=ChatOpenAI(model_kwargs={"engine":"gpt-4","messages":{"role": "system","content": system_message}},temperature=0.0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


# Reading the data from csv

In [9]:
df=pd.read_csv("supermarket_sales - Sheet1.csv")

# Creating an instance of the agent that will answer questions regarding our data frame as data analytics expert

In [10]:
agent=create_pandas_dataframe_agent(chat,df,verbose=True,engine="gpt-4",model_kwargs={"engine":"gpt-4","messages":{"role": "system","content": system_message}},temperature=0.0)

# Ready to start asking analytics questions in natural language

As you can see, lanchin is verbose, explaining all the steps required to arrive to the correct answer 

In [11]:
question=input("enter your question ")

enter your question i want to know total sales for each city, not including taxes


In [ ]:
#prompt=system_message+"\n"+question

In [14]:
answer=agent.run(question)



> Entering new AgentExecutor chain...
Thought: To find the total sales for each city without taxes, I need to group the dataframe by the 'City' column and then sum the 'cogs' column for each group.
Action: python_repl_ast
Action Input: df.groupby('City')['cogs'].sum()
Observation: City
Mandalay     101140.64
Naypyitaw    105303.53
Yangon       101143.21
Name: cogs, dtype: float64
Thought:I now know the total sales for each city, not including taxes.
Final Answer: The total sales for each city are as follows: Mandalay - 101140.64, Naypyitaw - 105303.53, and Yangon - 101143.21.

> Finished chain.


In [15]:
print(answer)

The total sales for each city are as follows: Mandalay - 101140.64, Naypyitaw - 105303.53, and Yangon - 101143.21.


Another example

In [16]:
question="show total sales by city and gender"

In [17]:
#prompt=system_message+"\n"+question

In [18]:
answer=agent.run(question)



> Entering new AgentExecutor chain...
Thought: To show total sales by city and gender, I need to group the dataframe by 'City' and 'Gender' columns and then sum the 'Total' column for each group.
Action: python_repl_ast
Action Input: df.groupby(['City', 'Gender'])['Total'].sum()
Observation: City       Gender
Mandalay   Female    52928.2950
           Male      53269.3770
Naypyitaw  Female    61685.4630
           Male      48883.2435
Yangon     Female    53269.1670
           Male      52931.2035
Name: Total, dtype: float64
Thought:I now know the total sales by city and gender.
Final Answer: 
Mandalay:
- Female: 52928.2950
- Male: 53269.3770

Naypyitaw:
- Female: 61685.4630
- Male: 48883.2435

Yangon:
- Female: 53269.1670
- Male: 52931.2035

> Finished chain.


In [19]:
print(answer)

Mandalay:
- Female: 52928.2950
- Male: 53269.3770

Naypyitaw:
- Female: 61685.4630
- Male: 48883.2435

Yangon:
- Female: 53269.1670
- Male: 52931.2035


# In next example you ask result in a Dictionary format, that can be used later on in your application

In [20]:
question="create a dictionary with the total sales per city, not including taxes, where the keys are the city name and the value is the total sales of the city"

In [21]:
#prompt=system_message+"\n"+question

In [22]:
result=agent.run(question)



> Entering new AgentExecutor chain...
Thought: To create a dictionary with the total sales per city, I need to group the dataframe by the 'City' column and then sum the 'cogs' column for each city.
Action: python_repl_ast
Action Input: df.groupby('City')['cogs'].sum()
Observation: City
Mandalay     101140.64
Naypyitaw    105303.53
Yangon       101143.21
Name: cogs, dtype: float64
Thought:I now have the total sales per city, not including taxes. I can create a dictionary with this information.
Final Answer: {'Mandalay': 101140.64, 'Naypyitaw': 105303.53, 'Yangon': 101143.21}

> Finished chain.


### Transforming result in a Python Dictionary

In [23]:
result=result.replace("'", "\"")

In [24]:
result=json.loads(result)

In [25]:
type(result)

dict

In [26]:
for city in result:
    print(city,result[city])

Mandalay 101140.64
Naypyitaw 105303.53
Yangon 101143.21


### As you can see above, answer is a dictionary that can be further used in the application